Complete algorithm:

  1. read csv with stations list as a dataframe
  2. read stations coordinates as list
  3. Create graph using bbox.
      Algorithm to create bbox:
        
        3.1. create linestring from points as input
        
        3.2. convert projection system of linestring into metres (EPSG 3857)
        
        3.3. create bbox with buffer of 100km
        
        3.4. convert projection back to (EPSG 4326)
        
        3.5. Get graph using bbox EPSG 4326

  4. inverse zip lat and longs into separate lists required for input to find nearest nodes on graphs
  5. find nearest nodes on graph
  6. iterate
      
      6.1. obtain list of nodes on the shortest path between 2 consecutive nodes
      
      6.2. construct linestring geometry using the list of nodes in 6.1 and append into a list
      
  7. linemerge the list of linestring

In [1]:
import geopandas as gpd
import osmnx as ox
import networkx as nx
from osgeo import ogr, osr
import pandas as pd
from shapely.geometry import MultiLineString, LineString
from shapely.ops import linemerge
import shapely.wkt
import os
import folium

In [2]:
def tupled(lat,long):
    cordtuple = (lat, long)
    return cordtuple

def node_list_to_path(G, node_list):
    """
    Source: https://towardsdatascience.com/find-and-plot-your-optimal-path-using-plotly-and-networkx-in-python-17e75387b873
    Given a list of nodes, return a line that
    follows the path
    defined by the list of nodes.
    Parameters
    ----------
    G : networkx multidigraph
    route : list
        the route as a list of nodes
    Returns
    -------
    linestring object
    """
    edge_nodes = list(zip(node_list[:-1], node_list[1:]))
    print("edge_nodes",edge_nodes)
    lines = []
    for u, v in edge_nodes:
        # if there are parallel edges, select the shortest in length
        data = min(G.get_edge_data(u, v).values(), key=lambda x: x['length'])
        
        # if it has a geometry attribute
        if 'geometry' in data:
            # add them to the list of lines to plot
            lines.append(data['geometry'])
            
        else:
            # if it doesn't have a geometry attribute,
            # then the edge is a straight line from node to node
            
            x1 = G.nodes[u]['x']
            y1 = G.nodes[u]['y']
            x2 = G.nodes[v]['x']
            y2 = G.nodes[v]['y']
            
            line = LineString([(x1, y1), (x2, y2)])
            lines.append(line)
    
    #lines is a list of linestrings. linemerge combines all the linestrings
    linegeom = linemerge(lines)
    return linegeom

def get_4326_bbox(listofstations):
    """
    Given a list of tuples (lat,long)
    returns a tuple of minx,maxs for the bounding box 
    for the linestring geometry constructed from the tuples
    with a buffer of 100 km
    in EPSG: 4326 CRS. 
    """
    
    lat_list, long_list = zip(*listofstations)
    new_list = zip(long_list,lat_list) #input is lat,long - Linestring takes long, lat
    
    raw_line = LineString(new_list)
    
    SpatialRef_4326 = osr.SpatialReference()
    SpatialRef_4326.ImportFromEPSG(4326)
    SpatialRef_32643 = osr.SpatialReference()
    SpatialRef_32643.ImportFromEPSG(32643)
    coordTransform = osr.CoordinateTransformation(SpatialRef_4326, SpatialRef_32643)
    geom = ogr.CreateGeometryFromWkt(raw_line.wkt)
    geom.Transform(coordTransform)
    
    bbox_32643 = geom.GetEnvelope()
    line = shapely.wkt.loads(geom.ExportToWkt())
    bbox_32643_buffered = (bbox_32643[0]-100000, bbox_32643[1]+100000, bbox_32643[2]- 100000, bbox_32643[3]+100000)
    
    #converting the buffered bbox back to 4326 points
    coordTransform_reverse = osr.CoordinateTransformation(SpatialRef_32643, SpatialRef_4326)

    #creating point geometry to be able to apply coordinate transforms on to the metres points
    string = 'POINT'+ '('+ str((bbox_32643_buffered[0])) + ' '+ str((bbox_32643_buffered[2]))+ ')'
    geom1 = ogr.CreateGeometryFromWkt(string)
    geom1.Transform(coordTransform_reverse)

    string2 = 'POINT'+ '('+ str((bbox_32643_buffered[1])) + ' '+ str((bbox_32643_buffered[3]))+ ')'
    geom2 = ogr.CreateGeometryFromWkt(string2)
    geom2.Transform(coordTransform_reverse)
    bbox_4326 = (geom1.GetY(), geom2.GetY(), geom2.GetX(), geom1.GetX())
    
    return bbox_4326


def route_extract(G, stations_list):
    #reading lats into one list and longs into a separate one
    lat_list, long_list = zip(*stations_list)

    #calculate nearest graph nodes to the stations
    nearest_nodes_to_stations = ox.distance.nearest_nodes(G,long_list,lat_list, return_dist = False )
    pairs_consec_stations = list(zip(nearest_nodes_to_stations[:-1],nearest_nodes_to_stations[1:]))

    #constructing the route
    lisofnodes = []
    for r, s in pairs_consec_stations:
        shortest_p = nx.shortest_path(G,r,s)
        #shortest_path returns a list of node IDs along shortest route on graph
        lisofnodes = lisofnodes + shortest_p[1:]
    print("inside route_extractmethod",lisofnodes)
    route = node_list_to_path(G,lisofnodes) 
    #LineString geometry is returned
    #node_list_to_path is defined separately

    return route
    

In [3]:
import os
import time

start = time.time()
dir_with_routes = "F:/Projects/IndianRailways_RoutesExtraction/routes/"
listicle_routes = []
listicle_graphs = []

for filename in os.listdir(dir_with_routes):
    
    try:
        if filename.endswith(".csv"): 
            path = dir_with_routes + filename
            df = pd.read_csv(path)
            print(path)

            #creating new column with tuple items = lat,long
            df['coords'] = df.apply(lambda x: tupled(x['stop_lat'],x['stop_lon']), axis = 1)
            stations_list = list(df.coords) 
            print(stations_list)

            #insert code for bbox and graph
            n,s,e,w = get_4326_bbox(stations_list) 
            print(n,s,e,w)
            G = ox.graph_from_bbox(n,s,e,w, custom_filter='["railway"~"rail"]')
            listicle_graphs.append(G)

            route_line = route_extract(G, stations_list)
            print(route_line)
            listicle_routes.append(route_line)

            df = pd.DataFrame({'geometry': [route_line]})
            gdf = gpd.GeoDataFrame(df, geometry =df['geometry'])

            save_path = filename + '.shp'
            gdf.to_file(save_path)
            

        else:
            continue
            
    except Exception as es:
        print(es)
        
        
end = time.time() - start
end

F:/Projects/IndianRailways_RoutesExtraction/routes/01028.csv
[(17.66864, 75.319433), (18.091768, 75.417538), (18.260653, 75.162191), (18.463897, 74.578972), (18.48208, 74.37667), (18.494017, 74.1366), (18.529178, 73.872886), (18.639373, 73.791862), (18.735029, 73.672214), (19.235126, 73.13015), (19.185887, 72.975547), (19.017189, 72.842982)]
19.669654876118663 16.92972498122377 76.65161040871166 71.63225908604252
inside route_extractmethod [6137852574, 5903261322, 6161200233, 1806196143, 2407893157, 1806196146, 2407893179, 5903870147, 5903870169, 2407893219, 2407893230, 2515163075, 2515169692, 2515169699, 2515169760, 2515131342, 6161155970, 2515131352, 7265593578, 7265606092, 7265606093, 7263610393, 7263610242, 7263610393, 7263610394, 7263610416, 2515085119, 6588186379, 6588186380, 2552632360, 2552632369, 2552632409, 2552632530, 2552632537, 2552632576, 2552632687, 2552632686, 2552632679, 2539975418, 2539975449, 1645591807, 2539975618, 2539975647, 2254876707, 2254876667, 2254876679, 225

F:/Projects/IndianRailways_RoutesExtraction/routes/02926.csv
[(31.63319, 74.86723), (31.519854, 75.291289), (31.331425, 75.591259), (31.306715, 75.632586), (31.217059, 75.765367), (30.912387, 75.847893), (30.701897, 76.822), (30.338658, 76.826706), (29.9691, 76.852455), (29.694316, 76.969571), (29.389018, 76.963863), (28.989073, 77.016993), (28.66855, 77.20041), (28.64179, 77.22175), (28.411483, 77.307358), (27.47897, 77.67335), (27.23635, 77.4888), (26.91604, 77.29644), (26.75186, 77.02834), (26.46822, 76.72679), (26.01924, 76.35685), (25.223733, 75.88068), (24.644326, 75.939131), (24.191271, 75.642972), (23.45616, 75.412731), (23.34068, 75.051126), (22.907329, 74.539704), (22.843907, 74.253674), (22.77684, 73.60479), (22.310696, 73.181005), (21.70342, 72.99955), (21.206418, 72.840643), (20.946371, 72.913814), (20.60744, 72.93353), (20.37348, 72.90847), (19.991417, 72.743568), (19.229149, 72.856822), (19.117139, 72.846563), (19.062341, 72.840965)]
22.826040356114483 27.260147795199845

F:/Projects/IndianRailways_RoutesExtraction/routes/43766.csv
[(13.116094, 79.913435), (13.123134, 79.936883), (13.12503, 79.965734), (13.123617, 79.998407), (13.123784, 80.026989), (13.121702, 80.043756), (13.118657, 80.064985), (13.118627, 80.074869), (13.118017, 80.100288), (13.116322, 80.126796), (13.115384, 80.136981), (13.114171, 80.153418), (13.114631, 80.166137), (13.113621, 80.183646), (13.109992, 80.209551), (13.108498, 80.226216), (13.107281, 80.238886), (13.1074, 80.244913), (13.109196, 80.25722), (13.108773, 80.28257), (13.103973, 80.293622), (13.09043, 80.291691), (13.08324, 80.282636), (13.079587, 80.276295), (13.073795, 80.273861), (13.062655, 80.280755), (13.055935, 80.280604), (13.045537, 80.276989), (13.035176, 80.267388), (13.028093, 80.260618), (13.021585, 80.252729), (13.013718, 80.248452), (13.006245, 80.247791), (12.988982, 80.251587), (12.978658, 80.241644), (12.976058, 80.232192), (12.967382, 80.219507)]
14.923132469060183 10.674061541469218 81.48853539565623 7

inside route_extractmethod [7633283627, 7633283656, 7633283654, 6282888921, 2525304933, 2525304935, 6282888892, 7633283714, 6282888892, 2525304944, 2525304946, 7633283723, 7633283724, 7633283723, 2525305087, 2525305116, 2525305159, 2525305196, 2525305199, 2525305248, 2525305269, 2525305289, 2525305299, 2525305518, 2525305520, 735163038, 4019482956, 4019482962, 4019482964, 4019482972, 4019484217, 4019484219, 4019484221, 6996401582, 6996401583, 4021042857, 6996401543, 6996401541, 3100101258, 4021042856, 4021042852, 4021042838, 6996418200, 4021042842, 4021042825, 4021042824, 4021042820, 6994505367, 6994505377, 6994505313, 6994505320]
edge_nodes [(7633283627, 7633283656), (7633283656, 7633283654), (7633283654, 6282888921), (6282888921, 2525304933), (2525304933, 2525304935), (2525304935, 6282888892), (6282888892, 7633283714), (7633283714, 6282888892), (6282888892, 2525304944), (2525304944, 2525304946), (2525304946, 7633283723), (7633283723, 7633283724), (7633283724, 7633283723), (7633283723

F:/Projects/IndianRailways_RoutesExtraction/routes/75240.csv
[(25.858374, 85.78743), (25.780648, 85.799875), (25.724215, 85.806227), (25.664949, 85.841665), (25.63101, 85.86797), (25.581378, 85.900262), (25.53223, 85.93147), (25.51057, 85.941324), (25.47435, 85.96512), (25.462088, 85.989068)]
26.791379546312353 23.398059275030747 87.24878524062983 84.52565369241279
inside route_extractmethod [6575959861, 6103699197, 6575105272, 6575105280, 6575113218, 6575113189, 6575113219, 5768131187, 5768131188, 6238474791, 6238474827, 6238474830, 6238474809, 6238474876, 6238474880, 6238474879, 6577470854, 5768131220, 6577470854, 6238474879, 6238474880, 6577470838, 5030727224, 5030727221, 5030724916, 5030724914, 5030724912, 5030724903, 1156483735, 1156770746, 1156772375, 1156772118, 1156483569, 1156772265, 1156771886, 1156483915]
edge_nodes [(6575959861, 6103699197), (6103699197, 6575105272), (6575105272, 6575105280), (6575105280, 6575113218), (6575113218, 6575113189), (6575113189, 6575113219), (657

1475.1515600681305